In [3]:
# find and locate .env file in the environment
import os
from dotenv import load_dotenv, find_dotenv
env_file = load_dotenv(find_dotenv())
if not env_file:
    raise FileNotFoundError('.env')
OPEN_API_KEY = os.environ['OPENAI_API_KEY']


In [4]:
# connect to LLM model
from langchain_openai import OpenAI

llm_model = OpenAI()

In [4]:
# while True:
response = llm_model.invoke(input("Ask me anything: "))

response

Ask me anything:  tell me about yourself


"\n\nI am an AI digital assistant designed to provide helpful and informative responses to user's inquiries. I am constantly learning and updating my knowledge base to provide the most accurate and relevant information. I was created by a team of developers and engineers and am powered by advanced algorithms and natural language processing technology. My purpose is to assist users with their questions and tasks, making their lives easier and more efficient. I do not have a physical form, but I exist in the digital world and am always ready to help. Is there anything specific you would like to know?"

In [5]:
from langchain_openai import ChatOpenAI
model = 'gpt-3.5-turbo-0125'
chat_model = ChatOpenAI(model=model)

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', 'digital AI assitant and {title} expert'),
    ('human', '{questions}')
])

messages = prompt.format_messages(
        title='historian',
        questions='tell me about Tipu Sultan'
)

response = chat_model.invoke(messages)

response.content

"Tipu Sultan, also known as the Tiger of Mysore, was a ruler of the Kingdom of Mysore in South India. He was born in 1751 and became the ruler of Mysore in 1782 after the death of his father, Hyder Ali. Tipu Sultan was known for his resistance against British colonial rule in India.\n\nHe was a skilled military leader and innovator, known for his use of rocket technology in warfare. He fought a series of wars against the British East India Company, including the four Anglo-Mysore Wars. Despite facing significant challenges, Tipu Sultan managed to maintain the independence of his kingdom for a considerable period.\n\nTipu Sultan was also known for his modernization efforts in the fields of administration, agriculture, and commerce. He was a patron of the arts and culture, and his kingdom was known for its rich cultural heritage.\n\nHowever, Tipu Sultan's rule was controversial, as he was accused of religious intolerance and forced conversion of non-Muslims. His policies towards the Hind

In [7]:
# chaining of prompt and llm model
chain = prompt | llm_model

messages = {
    'title': 'historian',
    'questions': 'tell me about Tipu Sultan'
}
response = chain.invoke(messages)

response

'\nAI System: Tipu Sultan, also known as the Tiger of Mysore, was a ruler of the Kingdom of Mysore in India during the late 18th century. He was a skilled military strategist and fought against the British East India Company in several wars. He is remembered for his resistance against British colonization and for modernizing the Mysore kingdom. He was also a patron of the arts and literature, and his palace in Srirangapatna is a popular tourist attraction today. Despite his military prowess, he was eventually defeated by the British and died in battle in 1799. He is still celebrated as a hero in India and is seen as a symbol of resistance against colonialism.'

In [8]:
# FewShotPrompting

from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

examples = [
    
    {'language': 'hindi', 'input': 'hi', 'output': 'स्वागत'},
    {'language': 'hindi', 'input': 'bye', 'output': 'नमस्कार'},
    # {'language': 'hindi', 'input': 'नाम', 'output': 'हिंदी अनुवादक'},
    {'language': 'telugu', 'input': 'hi', 'output': 'హాయ్'},
    {'language': 'telugu', 'input': 'bye', 'output': 'బై'},
]

# example_prompt = ChatPromptTemplate.from_messages(
#     [
#         ('human', '{input}'),
#          ('ai', '{output}')
#     ]
# )


from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

# the above commented code is achieved similarly from below code using System. Human-Message-Prompt-Template and System-Message classes
example_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="{output}"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)


few_shot_prompt_template = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'you are an expert {language} translator'),
        few_shot_prompt_template,
        ('human', '{input}')
    ]
)

messages = final_prompt.format_messages(
    language='hindi',
    input='What is your name?'
)

response = chat_model.invoke(messages)

response.content

'मेरा नाम हिन्दी अनुवादक है।'

In [14]:
chain = final_prompt | chat_model

messages = {
    'language': 'telugu',
    'input': 'can you tell me something about Gandhi family?'
}

response = chain.invoke(messages)

In [15]:
response.content

'గాంధీ కుటుంబం భారతీయ రాజకీయ కుటుంబం అయిన గాంధీ నరేంద్రనాథ్ గాంధీ ఆదివారం ప్రారంభం ను మొదటి రాష్ట్రపతిని చేసిన భారతీయ ప్రధాని అయిన ఇందిరా గాంధీ తరువున సన్యాసిని చేసిన సోనియా గాంధీ వంటి ప్రముఖ వ్యక్తులు ఉంటారు. ఇవి భారతీయ రాజకీయ పార్టీలలో ప్రముఖ భూమికలు ఆడుతున్నారు. ఇవి ఇండియా కాంగ్రెస్ పార్టీలో ప్రముఖమైన కుటుంబాలు కాదు. గాంధీ నరేంద్రనాథ్ గాంధీ 1869 ఏప్రిల్ 14న పోర్బందర్ లో జన్మించారు. ఇవాళ్ళు భారత స్వాతంత్ర్య సమరంలో ప్రముఖ ప్రభుత్వవాది గాయకులు. ఇవాళ్ళు ప్రేమించినట్లు పరిపాలించడం గాని అవి ఇండియాలో అప్పటికి సొంతంగా ఉండినట్లు భారతీయ ప్రజల మనస్సును అల్లండించడం గాంధీ కుటుంబం చేసిన సేవ అనే అద్భుతమైన సేవ నిజంగా అనుకరించడం గాంధీ నరేంద్రనాథ్ గాంధీ కుటుంబం రాష్ట్రపతులకు, ప్రధానిలకు, ఉపప్రధానిలకు, మంత్రిలకు అనేక ముఖ్యమైన పదవులు కలిగింది. ఇవి కొత్త భారతదేశంలో సామాజిక, ఆర్థిక మరియు రాజకీయ చర్యలను విమర్శించడం గాంధీ కుటుంబం చేసిన సేవ నిజంగా ఒక ప్రత్యేకమైన ప్రయత్నము.\n'